In [1]:
import sys
import os
import re
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas

Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple
Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple


In [216]:
import pandas as pd
import numpy as np
from functools import reduce
from collections import defaultdict 


IGNORED_FILES = ['.DS_Store']
RAW_DATA_ROOT_PATH = 'data/kaggle_raw/'



In [241]:
"""

write_data_paths : writes a list of paths to files in data directory

    individual data directories follow a naming convention in terms of {source} followed by {year} (e.g. WICAgencies2013)
    
    individual data files are named by the dimension they contain (e.g. Food_Costs.csv)

    output: a list of directory paths to data files

"""
def write_data_paths():
    data_paths = []
    for subdir, dirs, files in os.walk(RAW_DATA_ROOT_PATH):
        for file in files:
            if file not in IGNORED_FILES:
                data_paths.append(os.path.join(subdir, file))
    return data_paths



"""
create_WIC_df : creates dataframe w/ State, Year, Month row entries of WIC data.

    multiple data dimension sharing common State, Year, Month are joined into single row 
    
    final outputs appends all rows into single WIC dataframe 
    
    Output: WIC dataframe

"""
def create_WIC_df(data_paths):

    # Store result groups into lists of subdataframes sharing common State, Year, Month row entries for merging
    WIC_df_dict = defaultdict(list) 
    
    # Filter to WIC data sources
    WIC_dir_re = re.compile('data\/kaggle_raw\/WICAgencies*.{4}ytd\/.*\.csv')
    WIC_data_paths = [path for path in data_paths if re.findall(WIC_dir_re, path)]
    
    
    for path in WIC_data_paths:
        
        # Preprocessing fields for dataframe row format following State, Year, Month
        _variables = [col for col in list(pd.read_csv(path, nrows=1)) if col != 'State Agency or Indian Tribal Organization']
        _subtitle = re.findall(re.compile('data\/kaggle_raw\/WICAgencies*.{4}ytd\/(.*)\.csv'),path)[0]
        _year = re.findall(re.compile('data\/kaggle_raw\/WICAgencies*(.{4})ytd\/.*\.csv'),path)[0]
        _WIC_df = pd.read_csv(path).melt(id_vars=['State Agency or Indian Tribal Organization'],value_vars=_variables).rename(columns={'variable':'month', 'value':_subtitle})
        _WIC_df['MONTH'] = _WIC_df['month'].str.extract(r'(^\d{4}\-\d{2}\-\d{2})')
        _WIC_df['YEAR'] = _WIC_df['MONTH'].str.extract(r'(^\d{4})')
        del _WIC_df['month']
        _WIC_df.dropna(inplace=True)
        
        if _WIC_df.empty:
            pass
        else:
            WIC_df_dict[_year].append(_WIC_df)


        
    # Merge 
    WIC_df = pd.concat([reduce(lambda x, y: pd.merge(x, y, how='outer', on = ['State Agency or Indian Tribal Organization','YEAR','MONTH']), WIC_df_dict[key]) for key in WIC_df_dict.keys()], axis=0)
    WIC_df.to_csv('test.csv')

    return WIC_df
    


In [242]:
data_paths = write_data_paths()
WIC_df = create_WIC_df(data_paths)



In [243]:
WIC_df



,State Agency or Indian Tribal Organization,Infants_Partially_Breastfed,MONTH,YEAR,Total_Infants,Total_Breastfeeding_Women,Total_Number_of_Participants,Pregnant_Women_Participating,Postpartum_Women_Participating,Average_Food_Cost_Per_Person,Infants_Fully_Breastfed,Food_Costs,Children_Participating,Women_Partially_Breastfeeding,Infants_Fully_Formula-fed,Rebates_Received,Women_Fully_Breastfeeding,Total_Women
0,Connecticut,3463.0,2014-10-01,2014,13408,3136,54403,5834,3001,48.0512,998.0,2.61413e+06,29024,2077.0,8947.0,1.00978e+06,1059.0,11971
1,Maine,589.0,2014-10-01,2014,5374,1454,22436,1899,1448,47.1698,964.0,1.0583e+06,12261,523.0,3821.0,312539,931.0,4801
2,Massachusetts,6063.0,2014-10-01,2014,26538,8866,115004,10946,7014,42.1583,3327.0,4.84837e+06,61640,5651.0,17148.0,1.91385e+06,3215.0,26826
3,New Hampshire,342.0,2014-10-01,2014,3706,892,14835,1408,1148,17.8349,636.0,264581,7681,277.0,2728.0,511446,615.0,3448
4,New York,37885.0,2014-10-01,2014,110153,48151,485855,44158,24290,56.0285,9440.0,2.72217e+07,259103,37596.0,62828.0,8.24818e+06,10555.0,116599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,Washington,5823.0,2016-09-01,2016,34573,15390,166122,14785,7997,44.47,9425.0,7.38745e+06,93377,5657.0,19325.0,1.7738e+06,9733.0,38172
1088,Northern Marianas,167.0,2016-09-01,2016,580,245,3256,267,130,70.317,74.0,228952,2034,170.0,339.0,41616,75.0,642
1089,"Inter-Tribal Council, AZ",231.0,2016-09-01,2016,2175,473,9499,775,643,39.0487,332.0,370924,5433,197.0,1612.0,168547,276.0,1891
1090,"Navajo Nation, AZ",398.0,2016-09-01,2016,1732,732,8290,632,317,29.4803,384.0,244392,4877,374.0,950.0,240639,358.0,1681
